In [50]:
import pandas as pd
from collections import Counter
import re
import json
import os
import numpy as np
from tqdm import tqdm
from scipy.stats import spearmanr

In [14]:
freq_df = pd.read_csv("data/frequency.csv", index_col=0)
freq_df.head()

,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
profession_merge,,,,,,,,,,,,,,,,,,,,,
accountant,0.000014,2.912382e-06,0.000008,2.953883e-06,0.000004,2.389870e-06,0.000012,1.118629e-05,0.000005,1.126231e-05,...,5.670011e-06,5.475462e-06,4.684715e-06,6.246820e-06,5.750266e-06,5.348605e-06,4.947257e-06,4.641961e-06,6.724351e-06,4.120042e-06
acrobat,0.000005,4.853971e-07,0.000002,9.846275e-07,0.000002,6.828201e-07,0.000001,6.292289e-06,0.000003,6.624888e-07,...,8.015198e-07,3.716377e-07,6.417418e-07,4.656015e-07,5.124000e-07,3.650925e-07,6.184072e-07,7.320016e-07,4.854478e-07,7.704143e-07
actor,0.000030,2.329906e-05,0.000041,3.544659e-05,0.000033,1.707050e-05,0.000021,3.950159e-05,0.000029,1.755595e-05,...,2.977498e-05,3.017698e-05,2.708150e-05,2.692729e-05,2.856155e-05,2.893358e-05,2.731905e-05,2.754826e-05,3.315429e-05,2.003077e-05
actress,0.000024,1.601810e-05,0.000016,4.676981e-05,0.000019,1.024230e-05,0.000016,1.398287e-05,0.000014,1.258729e-05,...,1.175562e-05,1.283389e-05,1.050317e-05,1.247424e-05,1.254431e-05,1.186551e-05,1.098582e-05,8.569775e-06,1.181256e-05,8.575046e-06
administrator,0.000005,9.707941e-07,0.000006,4.923138e-07,0.000001,2.048460e-06,0.000004,6.991433e-07,0.000001,1.656222e-06,...,2.048329e-06,2.130723e-06,1.946617e-06,1.998206e-06,2.125511e-06,1.898481e-06,1.818845e-06,1.874638e-06,1.833914e-06,1.574325e-06


In [16]:
df = freq_df.T

In [17]:
df.columns

Index(['accountant', 'acrobat', 'actor', 'actress', 'administrator', 'admiral',
       'advertiser', 'adviser', 'airman', 'alderman',
       ...
       'warden', 'watcher', 'watchman', 'weatherman', 'weaver',
       'wedding planner', 'wingman', 'witch doctor', 'wrestler', 'writer'],
      dtype='object', name='profession_merge', length=500)

In [19]:
df.index.name

In [21]:
df.Name

AttributeError: 'DataFrame' object has no attribute 'Name'

In [22]:
df.to_csv("data/frequency.csv", index=True)

In [23]:
df.head()

profession_merge,accountant,acrobat,actor,actress,administrator,admiral,advertiser,adviser,airman,alderman,...,warden,watcher,watchman,weatherman,weaver,wedding planner,wingman,witch doctor,wrestler,writer
1950,0.000014,5.288248e-06,0.000030,0.000024,5.288248e-06,0.000013,4.807498e-07,4.807498e-06,4.807498e-07,9.614996e-07,...,0.000025,0.000000e+00,0.000009,9.614996e-07,2.403749e-06,0.0,0.000000e+00,0.000001,0.000010,0.000022
1951,0.000003,4.853971e-07,0.000023,0.000016,9.707941e-07,0.000019,0.000000e+00,1.456191e-06,0.000000e+00,4.853971e-07,...,0.000003,9.707941e-07,0.000006,0.000000e+00,9.707941e-07,0.0,4.853971e-07,0.000002,0.000001,0.000020
1952,0.000008,2.494461e-06,0.000041,0.000016,5.986707e-06,0.000011,2.993353e-06,9.977844e-07,0.000000e+00,9.977844e-07,...,0.000005,4.988922e-07,0.000001,0.000000e+00,1.496677e-06,0.0,0.000000e+00,0.000001,0.000004,0.000017
1953,0.000003,9.846275e-07,0.000035,0.000047,4.923138e-07,0.000022,0.000000e+00,4.430824e-06,4.923138e-07,9.846275e-07,...,0.000020,0.000000e+00,0.000003,1.476941e-06,9.846275e-07,0.0,0.000000e+00,0.000000,0.000001,0.000009
1954,0.000004,1.868240e-06,0.000033,0.000019,1.401180e-06,0.000050,0.000000e+00,2.802361e-06,0.000000e+00,9.341202e-07,...,0.000023,0.000000e+00,0.000009,4.670601e-07,9.341202e-07,0.0,9.341202e-07,0.000000,0.000002,0.000021


In [24]:
df = pd.read_csv("sentiment-profession/top500_merged_profession_sentiment.csv", index_col=None)
df.head()

,profession,year,sentiment,imdb
0,doctor,1950,-1,31
1,doctor,1950,0,1398
2,doctor,1950,1,112
3,doctor,1951,-1,19
4,doctor,1951,0,1209


In [25]:
df.profession.unique().size

500

In [28]:
n, n_pos_gt_neg = 0, 0
for _, subdf in df.groupby(["profession","year"]):
    n += 1
    n_pos_gt_neg += subdf.loc[subdf.sentiment == 1, "imdb"].item() > subdf.loc[subdf.sentiment == -1, "imdb"].item()

In [29]:
n, n_pos_gt_neg

(34000, 16190)

In [34]:
professions = sorted(df["profession"].unique())
recs = []
header = ["year"]

for year in range(1950, 2018):
    rec = [year]
    for profession in professions:
        pos = df.loc[(df.year == year) & (df.profession == profession) & (df.sentiment == 1), "imdb"].item()
        neg = df.loc[(df.year == year) & (df.profession == profession) & (df.sentiment == -1), "imdb"].item()
        if pos + neg == 0:
            posf, negf = 0, 0
        else:
            posf = pos/(pos + neg)
            negf = neg/(pos + neg)
        rec.append(posf)
        rec.append(negf)
    recs.append(rec)

for profession in professions:
    header.append("{}-pos".format(profession))
    header.append("{}-neg".format(profession))

sdf = pd.DataFrame(recs, columns=header)

In [36]:
sdf = sdf.drop(columns=[profession + "-neg" for profession in professions])

In [38]:
sdf.columns = ["year"] + professions

In [40]:
sdf.index = sdf.year

In [43]:
sdf = sdf.drop(columns=["year"])

In [44]:
sdf.head()

,accountant,acrobat,actor,actress,administrator,admiral,advertiser,adviser,airman,alderman,...,warden,watcher,watchman,weatherman,weaver,wedding planner,wingman,witch doctor,wrestler,writer
year,,,,,,,,,,,,,,,,,,,,,
1950,1.0,1.0,0.750000,0.954545,0.0,0.666667,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.714286,0.769231
1951,1.0,0.0,0.785714,1.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.833333
1952,1.0,0.0,0.904762,0.900000,1.0,1.000000,1.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000,0.888889
1953,0.0,0.0,0.863636,0.812500,1.0,0.833333,0.0,0.333333,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.800000
1954,1.0,0.5,0.750000,0.888889,0.0,0.857143,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.833333


In [46]:
(sdf > 0.5).sum().sum()

16190

In [47]:
sdf.to_csv("sentiment-profession/sentiment.csv", index=True)

In [48]:
tdf = pd.read_csv("/home/saby/projects/profession/plos/data/release/soc-mapped-expanded-taxonomy.csv", index_col=None)

In [49]:
tdf.head()

,profession,sense-name,sense-definition,primary-soc,secondary-soc
0,accountant,accountant.n.01,someone who maintains and audits business acco...,13:Business and Financial Operations Occupations,NaN
1,acrobat,acrobat.n.01,an athlete who performs acts requiring skill a...,"27:Arts, Design, Entertainment, Sports, and Me...",NaN
2,actor,actor.n.01,a theatrical performer,"27:Arts, Design, Entertainment, Sports, and Me...",NaN
3,actor,actor.n.02,a person who acts and gets things done,"27:Arts, Design, Entertainment, Sports, and Me...",NaN
4,actress,actress.n.01,a female actor,"27:Arts, Design, Entertainment, Sports, and Me...",NaN


In [50]:
tdf["primary-soc"].unique()

array(['13:Business and Financial Operations Occupations',
       '27:Arts, Design, Entertainment, Sports, and Media Occupations',
       '11:Management Occupations', '41:Sales and Related Occupations',
       'General', '55:Military Specific Occupations',
       '29:Healthcare Practitioners and Technical Occupations',
       "Can't Say", '19:Life, Physical, and Social Science Occupations',
       '17:Architecture and Engineering Occupations',
       '33:Protective Service Occupations', '51:Production Occupations',
       '23:Legal Occupations',
       '53:Transportation and Material Moving Occupations',
       '39:Personal Care and Service Occupations',
       '35:Food Preparation and Serving Related Occupations',
       '21:Community and Social Service Occupations',
       '47:Construction and Extraction Occupations',
       '43:Office and Administrative Support Occupations',
       '45:Farming, Fishing, and Forestry Occupations',
       '37:Building and Grounds Cleaning and Maintena

In [51]:
tdf["sense-name"].unique().size, tdf["profession"].unique().size

(563, 500)

In [52]:
tdf.loc[tdf["primary-soc"] == "13:Business and Financial Operations Occupations"]

,profession,sense-name,sense-definition,primary-soc,secondary-soc
0,accountant,accountant.n.01,someone who maintains and audits business acco...,13:Business and Financial Operations Occupations,NaN
15,analyst,analyst.n.02,an expert who studies financial data (on credi...,13:Business and Financial Operations Occupations,NaN
34,auditor,auditor.n.03,a qualified accountant who inspects the accoun...,13:Business and Financial Operations Occupations,NaN
156,contractor,contractor.n.01,someone (a person or firm) who contracts to bu...,13:Business and Financial Operations Occupations,NaN
249,executive,executive.n.01,a person responsible for the administration of...,13:Business and Financial Operations Occupations,NaN
435,operator,operator.n.03,someone who owns or operates a business,13:Business and Financial Operations Occupations,NaN
525,recruiter,recruiter.n.01,someone who supplies members or employees,13:Business and Financial Operations Occupations,NaN
610,steward,shop_steward.n.01,a union member who is elected to represent fel...,13:Business and Financial Operations Occupations,NaN
676,wedding planner,NaN,NaN,13:Business and Financial Operations Occupations,NaN


In [72]:
taxonomy = {"name": "SOC", "children": []}

for soc, socdf in tdf.groupby("primary-soc"):
    if re.match("^\d", soc):
        soc = soc[3:]
        print(soc)
        soc_node = {"name": soc, "children": []}

        for (sense, sense_defn), sense_df in socdf.groupby(["sense-name", "sense-definition"], dropna=False):
            sense_name = sense if pd.notna(sense) else "none"
            sense_node = {"name": sense_name, "children": []}
            
            for profession in sense_df["profession"].unique():
                sense_node["children"].append({"name": profession, "value": 1})
            
            soc_node["children"].append(sense_node)

        taxonomy["children"].append(soc_node)

Management Occupations
Business and Financial Operations Occupations
Computer and Mathematical Occupations
Architecture and Engineering Occupations
Life, Physical, and Social Science Occupations
Community and Social Service Occupations
Legal Occupations
Educational Instruction and Library Occupations
Arts, Design, Entertainment, Sports, and Media Occupations
Healthcare Practitioners and Technical Occupations
Healthcare Support Occupations
Protective Service Occupations
Food Preparation and Serving Related Occupations
Building and Grounds Cleaning and Maintenance Occupations
Personal Care and Service Occupations
Sales and Related Occupations
Office and Administrative Support Occupations
Farming, Fishing, and Forestry Occupations
Construction and Extraction Occupations
Installation, Maintenance, and Repair Occupations
Production Occupations
Transportation and Material Moving Occupations
Military Specific Occupations


In [73]:
json.dump(taxonomy, open("taxonomy.json", "w"), indent=2)

In [62]:
sense_df

,profession,sense-name,sense-definition,primary-soc,secondary-soc
607,staff sergeant,staff_sergeant.n.01,a noncommissioned officer ranking above corpor...,55:Military Specific Occupations,NaN


In [3]:
profession_dir = "/home/saby/projects/profession/plos/data/analysis_data/media_data/profession/"

In [4]:
files = os.listdir(profession_dir)

In [5]:
len(files)

2170

In [6]:
professions = []
for file in files:
    if re.match("[^\.]+\.csv", file):
        professions.append(file.split(".")[0])
professions = sorted(professions)

In [7]:
len(professions)

500

In [8]:
n_frequency_files_exist = 0
n_sentiment_files_exist = 0

for profession in professions:
    frequency_path = os.path.join(profession_dir, "{}.freq.coeff.csv".format(profession))
    sentiment_path = os.path.join(profession_dir, "{}.sentiment.coeff.csv".format(profession))
    n_frequency_files_exist += os.path.exists(frequency_path)
    n_sentiment_files_exist += os.path.exists(sentiment_path)

print(n_frequency_files_exist)
print(n_sentiment_files_exist)

500
334


In [9]:
frequency_coeffs = set()
sentiment_coeffs = set()

for profession in professions:
    frequency_path = os.path.join(profession_dir, "{}.freq.coeff.csv".format(profession))
    sentiment_path = os.path.join(profession_dir, "{}.sentiment.coeff.csv".format(profession))
    if os.path.exists(frequency_path):
        frequency_df = pd.read_csv(frequency_path, index_col=None)
        frequency_coeffs.update(frequency_df.param.values)
    if os.path.exists(sentiment_path):
        sentiment_df = pd.read_csv(sentiment_path, index_col=None)
        sentiment_coeffs.update(sentiment_df.param.values)

frequency_coeffs = sorted(frequency_coeffs)
sentiment_coeffs = sorted(sentiment_coeffs)

print(frequency_coeffs)
print(sentiment_coeffs)

['C(Country_Canada)[T.1]', 'C(Country_Japan)[T.1]', 'C(Country_United_Kingdom)[T.1]', 'C(Country_United_States)[T.1]', 'C(Genre_Action)[T.1]', 'C(Genre_Adventure)[T.1]', 'C(Genre_Animation)[T.1]', 'C(Genre_Biography)[T.1]', 'C(Genre_Comedy)[T.1]', 'C(Genre_Crime)[T.1]', 'C(Genre_Documentary)[T.1]', 'C(Genre_Drama)[T.1]', 'C(Genre_Family)[T.1]', 'C(Genre_Fantasy)[T.1]', 'C(Genre_Game_Show)[T.1]', 'C(Genre_History)[T.1]', 'C(Genre_Horror)[T.1]', 'C(Genre_Music)[T.1]', 'C(Genre_Musical)[T.1]', 'C(Genre_Mystery)[T.1]', 'C(Genre_News)[T.1]', 'C(Genre_Reality_TV)[T.1]', 'C(Genre_Romance)[T.1]', 'C(Genre_Sci_Fi)[T.1]', 'C(Genre_Short)[T.1]', 'C(Genre_Sport)[T.1]', 'C(Genre_Thriller)[T.1]', 'C(Genre_War)[T.1]', 'C(Genre_Western)[T.1]', 'Intercept', 'kind[T.movie]', 'year']
['C(Country_Canada)[T.1]', 'C(Country_United_Kingdom)[T.1]', 'C(Country_United_States)[T.1]', 'C(Genre_Action)[T.1]', 'C(Genre_Adventure)[T.1]', 'C(Genre_Animation)[T.1]', 'C(Genre_Comedy)[T.1]', 'C(Genre_Crime)[T.1]', 'C(Ge

In [14]:
frequency_data = []
sentiment_data = []

for profession in tqdm(professions):
    frequency_path = os.path.join(profession_dir, "{}.freq.coeff.csv".format(profession))
    sentiment_path = os.path.join(profession_dir, "{}.sentiment.coeff.csv".format(profession))
    frequency_arr = np.zeros(len(frequency_coeffs))
    sentiment_arr = np.zeros(len(sentiment_coeffs))

    if os.path.exists(frequency_path):
        frequency_df = pd.read_csv(frequency_path, index_col=None)
        for _, row in frequency_df.iterrows():
            param = row["param"]
            coeff = row["coefficient"]
            i = frequency_coeffs.index(param)
            frequency_arr[i] = coeff
    frequency_data.append(frequency_arr)
    
    if os.path.exists(sentiment_path):
        sentiment_df = pd.read_csv(sentiment_path, index_col=None)
        for _, row in sentiment_df.iterrows():
            param = row["param"]
            coeff = row["coefficient"]
            i = sentiment_coeffs.index(param)
            sentiment_arr[i] = coeff
    sentiment_data.append(sentiment_arr)

frequency_data = np.concatenate(frequency_data).reshape(-1, len(frequency_coeffs)).T
sentiment_data = np.concatenate(sentiment_data).reshape(-1, len(sentiment_coeffs)).T

frequency_df = pd.DataFrame(frequency_data, index=frequency_coeffs, columns=professions)
sentiment_df = pd.DataFrame(sentiment_data, index=sentiment_coeffs, columns=professions)

100%|██████████| 500/500 [00:01<00:00, 484.11it/s]


In [15]:
frequency_df.to_csv("media/profession.csv", index=True)
sentiment_df.to_csv("media/sentiment.csv", index=True)

In [16]:
frequency_df.max().max(), frequency_df.min().min()

(166.77403686589452, -576.1471339360988)

In [13]:
x = np.array([1,2,3,4,5,6])
print(x.reshape(-1,3).T)

[[1 4]
 [2 5]
 [3 6]]


In [2]:
soc = pd.read_csv("employment/soc_frequency.csv", index_col=None)
emp = pd.read_csv("employment/emp.csv", index_col=None)

In [3]:
soc.head()

,soc_code,soc_name,1950,1951,1952,1953,1954,1955,1956,1957,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,11,Management,0.000416,4.412259e-04,5.852006e-04,4.091884e-04,0.000544,0.000548,5.693715e-04,0.000561,...,0.000378,0.000436,0.000433,0.000387,0.000396,0.000405,0.000385,0.000405,0.000444,0.000436
1,13,Business and Financial Operations,0.000054,4.611272e-05,3.741692e-05,5.316989e-05,0.000040,0.000038,6.842440e-05,0.000046,...,0.000044,0.000050,0.000058,0.000052,0.000048,0.000048,0.000048,0.000045,0.000051,0.000045
2,15,Computer and Mathematical,0.000000,9.707941e-07,9.977844e-07,9.846275e-07,0.000001,0.000000,7.054062e-07,0.000001,...,0.000010,0.000004,0.000004,0.000005,0.000007,0.000008,0.000008,0.000012,0.000012,0.000008
3,17,Architecture and Engineering,0.000016,3.203621e-05,2.644129e-05,3.052345e-05,0.000022,0.000035,3.421220e-05,0.000045,...,0.000029,0.000023,0.000025,0.000021,0.000021,0.000020,0.000022,0.000022,0.000025,0.000032
4,19,"Life, Physical, and Social Science",0.000048,3.349240e-05,1.845901e-05,3.495428e-05,0.000031,0.000022,2.927436e-05,0.000036,...,0.000091,0.000086,0.000082,0.000065,0.000061,0.000061,0.000067,0.000056,0.000059,0.000055


In [4]:
emp.head()

,mcode,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,11,8063410,7782680,7212360,7092460,6653480,6200940,5960560,5892900,6003930,...,6116380,6022860,6183820,6390430,6542950,6741640,6936990,7090790,7280330,7616650
1,13,4361980,4619270,4676680,4772120,4924210,5131840,5410410,5826140,6015500,...,6063670,6090910,6178070,6419370,6658090,6828940,7032560,7281190,7472750,7721300
2,15,2620080,2932810,2825870,2772620,2827010,2915300,2952740,3076200,3191360,...,3303690,3283950,3406720,3578220,3696180,3834180,4005250,4165140,4261460,4384300
3,17,2506380,2575620,2489070,2411260,2376650,2372770,2382480,2430250,2486020,...,2412730,2305530,2310830,2356530,2380840,2418020,2475390,2499050,2516780,2556220
4,19,909530,1038670,1067730,1078630,1113130,1131390,1185730,1231070,1255670,...,1308380,1064510,1082370,1104100,1135030,1144440,1146110,1152840,1148300,1171910


In [11]:
media = soc.loc[soc.soc_code.isin(np.arange(22)*2 + 11), ["soc_name"] + np.arange(1999, 2018).astype(str).tolist()]

In [12]:
media.to_csv("employment/media.csv", index=False)

In [19]:
emp[np.arange(1999, 2019).astype(str)] /= emp[np.arange(1999, 2019).astype(str)].sum()

In [28]:
employment = emp[["mcode"] + np.arange(1999, 2018).astype(str).tolist()].copy()

In [29]:
employment.rename(columns={"mcode":"soc_name"}, inplace=True)

In [30]:
employment.shape

(22, 20)

In [31]:
media.shape

(22, 20)

In [33]:
employment["soc_name"] = media["soc_name"]

In [34]:
employment

,soc_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Management,0.063355,0.059987,0.056355,0.055617,0.052156,0.048397,0.045742,0.044440,0.044687,0.045513,0.046816,0.047388,0.048206,0.049049,0.049348,0.049891,0.050306,0.050504,0.051072
1,Business and Financial Operations,0.034272,0.035604,0.036542,0.037421,0.038601,0.040053,0.041520,0.043936,0.044773,0.045386,0.046412,0.047923,0.048161,0.049271,0.050216,0.050537,0.050999,0.051860,0.052422
2,Computer and Mathematical,0.020586,0.022605,0.022080,0.021742,0.022161,0.022753,0.022660,0.023198,0.023753,0.024472,0.025287,0.025838,0.026557,0.027464,0.027877,0.028374,0.029045,0.029666,0.029895
3,Architecture and Engineering,0.019693,0.019852,0.019449,0.018908,0.018630,0.018519,0.018283,0.018327,0.018503,0.018653,0.018467,0.018140,0.018014,0.018087,0.017957,0.017894,0.017951,0.017799,0.017656
4,"Life, Physical, and Social Science",0.007146,0.008006,0.008343,0.008458,0.008726,0.008830,0.009099,0.009284,0.009346,0.009593,0.010015,0.008376,0.008438,0.008474,0.008561,0.008469,0.008311,0.008211,0.008055
5,Community and Social Service,0.011036,0.011323,0.011907,0.012366,0.012665,0.013063,0.012992,0.013191,0.013346,0.013772,0.014476,0.014958,0.014737,0.014446,0.014343,0.014288,0.014302,0.014382,0.014709
6,Legal,0.006744,0.006867,0.007106,0.007331,0.007459,0.007481,0.007572,0.007366,0.007433,0.007421,0.007647,0.007810,0.007814,0.007852,0.007857,0.007792,0.007704,0.007660,0.007687
7,Educational Instruction and Library,0.057709,0.057430,0.059841,0.060949,0.061392,0.061593,0.061995,0.061886,0.061899,0.062516,0.064974,0.066546,0.065553,0.064280,0.063359,0.062428,0.061950,0.061513,0.061226
8,"Arts, Design, Entertainment, Sports, and Media",0.012191,0.011665,0.011789,0.011791,0.012057,0.012454,0.012918,0.013027,0.013109,0.013352,0.013362,0.013507,0.013453,0.013433,0.013261,0.013274,0.013369,0.013554,0.013505
9,Healthcare Practitioners and Technical Occupat...,0.047158,0.046564,0.047812,0.048501,0.048396,0.049633,0.050245,0.050630,0.051191,0.052349,0.055117,0.057803,0.058583,0.058716,0.058495,0.058125,0.058173,0.059249,0.059676


In [35]:
media

,soc_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Management,0.000442,0.000466,0.000429,0.000417,0.000431,0.000445,0.000454,0.000439,3.661659e-04,0.000378,0.000436,0.000433,0.000387,0.000396,4.049724e-04,0.000385,0.000405,4.441448e-04,4.360381e-04
1,Business and Financial Operations,0.000040,0.000040,0.000048,0.000045,0.000045,0.000053,0.000057,0.000047,5.039946e-05,0.000044,0.000050,0.000058,0.000052,0.000048,4.765027e-05,0.000048,0.000045,5.095036e-05,4.489468e-05
2,Computer and Mathematical,0.000003,0.000003,0.000009,0.000004,0.000005,0.000006,0.000006,0.000005,3.467538e-06,0.000010,0.000004,0.000004,0.000005,0.000007,8.269345e-06,0.000008,0.000012,1.157883e-05,7.938617e-06
3,Architecture and Engineering,0.000024,0.000026,0.000023,0.000019,0.000023,0.000020,0.000023,0.000024,2.403363e-05,0.000029,0.000023,0.000025,0.000021,0.000021,1.991580e-05,0.000022,0.000022,2.504551e-05,3.235740e-05
4,"Life, Physical, and Social Science",0.000056,0.000050,0.000057,0.000060,0.000054,0.000054,0.000059,0.000054,6.591312e-05,0.000091,0.000086,0.000082,0.000065,0.000061,6.111648e-05,0.000067,0.000056,5.899090e-05,5.543633e-05
5,Community and Social Service,0.000150,0.000158,0.000147,0.000175,0.000167,0.000152,0.000165,0.000162,1.397534e-04,0.000148,0.000147,0.000140,0.000131,0.000138,1.393368e-04,0.000148,0.000141,1.431005e-04,1.377357e-04
6,Legal,0.000222,0.000207,0.000214,0.000180,0.000193,0.000200,0.000208,0.000202,2.049545e-04,0.000184,0.000192,0.000188,0.000211,0.000216,1.892269e-04,0.000208,0.000198,1.954679e-04,2.045878e-04
7,Educational Instruction and Library,0.000150,0.000148,0.000150,0.000147,0.000153,0.000143,0.000130,0.000137,1.393891e-04,0.000141,0.000130,0.000143,0.000136,0.000150,1.361065e-04,0.000132,0.000118,1.183054e-04,1.041734e-04
8,"Arts, Design, Entertainment, Sports, and Media",0.000393,0.000439,0.000407,0.000426,0.000411,0.000462,0.000422,0.000443,4.370568e-04,0.000434,0.000453,0.000445,0.000449,0.000456,4.355932e-04,0.000409,0.000410,4.006259e-04,3.509781e-04
9,Healthcare Practitioners and Technical Occupat...,0.000502,0.000429,0.000489,0.000474,0.000470,0.000532,0.000505,0.000504,4.536222e-04,0.000449,0.000483,0.000437,0.000417,0.000419,4.397531e-04,0.000435,0.000437,4.251030e-04,4.249076e-04


In [48]:
for i in range(22):
    x = media.iloc[i, 1:]
    y = employment.iloc[i, 1:]
    res = spearmanr(x, y)
    print("{:50s} corr = {:8.4f} p = {:8.4f} media = [{:.2e}, {:.2e}] employment = [{:.2e}, {:.2e}]".format(media.iloc[i, 0], res.correlation, res.pvalue, x.min(), x.max(), y.min(), y.max()))

Management                                         corr =   0.2667 p =   0.2698 media = [3.66e-04, 4.66e-04] employment = [4.44e-02, 6.34e-02]
Business and Financial Operations                  corr =   0.2404 p =   0.3216 media = [3.96e-05, 5.84e-05] employment = [3.43e-02, 5.24e-02]
Computer and Mathematical                          corr =   0.5877 p =   0.0081 media = [2.51e-06, 1.20e-05] employment = [2.06e-02, 2.99e-02]
Architecture and Engineering                       corr =   0.0807 p =   0.7426 media = [1.90e-05, 3.24e-05] employment = [1.77e-02, 1.99e-02]
Life, Physical, and Social Science                 corr =   0.4105 p =   0.0808 media = [5.00e-05, 9.09e-05] employment = [7.15e-03, 1.00e-02]
Community and Social Service                       corr =  -0.7737 p =   0.0001 media = [1.31e-04, 1.75e-04] employment = [1.10e-02, 1.50e-02]
Legal                                              corr =  -0.0807 p =   0.7426 media = [1.80e-04, 2.22e-04] employment = [6.74e-03, 7.86e-03]

In [49]:
media.to_csv("employment/media.csv", index=False)
employment.to_csv("employment/employment.csv", index=False)

In [53]:
media = pd.read_csv("employment/media.csv", index_col=0)
employment = pd.read_csv("employment/employment.csv", index_col=0)

In [56]:
media.T.to_csv("employment/media.csv", index=True)
employment.T.to_csv("employment/employment.csv", index=True)